# 8 客户关系管理
在客户关系管理这一概念成为时髦术语之前，许多组织设计并开发了以客户为中心的维度模型，以便能够更好地理解客户的行为。客户关系管理是一种重要的商业策略，很多观念认为，客户关系管理是企业生存的基础  
本章首先综述客户关系管理的基本概念，包含其操作和分析角色。在描述各种不同的客户层次时，将探讨建模复杂层次存在的问题  
本章主要包含以下概念：  
+ 客户关系管理概述
+ 客户名称和地址解析，包括国际方面的考虑 5  
+ 处理日期、聚集事实，以及在客户维度中分割行为属性和积分  
+ 用于处理低粒度属性的支架表  
+ 稀疏数据的桥接表，以及权衡桥接表与位置设计  
+ 应用于多客户联系人的桥接表
+ 获取客户队列组的行为研究分组 5
+ 分析有序客户行为的步骤维度  
+ 带有有效日期和失效日期的包含时间范围的事实表
+ 完善事实表使用的满意维度或异常情况  
+ 在ETL过程下游通过主数据管理或部分一致性集成客户数据  
+ 有关对应表连接的警告 5  
+ 实现实时、低时间延迟需求的检查  

因为本章以客户为中心的建模问题和模式与跨行业和功能区的情况是相关的，因此不讨论总线矩阵 5  
## 8.1 客户关系管理概述
无论哪个行业，组织结构都蜂拥而至推崇客户关系管理这一概念。客户关系管理必须关注业务的各个方面，包括市场、销售、服务、操作等方面，建立并维持与客户之间的共同利益关系  
客户关系管理能够为接受它的企业带来明显的利益，不仅带来收入的增加，而且能够提高运作效率  
在许多组织中，对客户的理解根据产品链的业务单元、业务功能、地理位置的不同而存在差异  
过去几十年来，社会媒介、位置跟踪技术、网络使用监控、多媒体应用、传感器网络的爆炸性增长提供了海量的客户行为数据，即使街道企业也认识到适用它们可获得可操作的知识 5  
拒绝改变是人类的天性，毫不奇怪的是，与人有关的问题通常成为客户关系管理实现的难题  
在第17章中，将强调为建立DW/BI系统获得业务高层与IT管理部门支持的重要性  
### 操作型与分析型客户关系管理
可以说客户关系管理患有人格分裂症，因为它需要同时满足操作型和分析型需求  
从操作角度来看，客户关系管理需要同步客户面对的过程。不要将这些过程当成是独立的系统，客户关系管理的观念是将这些客户活动集成  
当数据在客户关系管理等价的操作端被建立后，显然需要存储和分析来自客户交互和交易系统的历史度量。数据仓库是支持客户全方位全景视图的基础 5  
分析型客户关系管理可确保获得DW/BI系统中精确的、集成的、可访问的客户数据。模型的结果也可以保留在DW/BI环境中以方便后续的分析  
![8-1](../syn_pic/the_datawarehouse_toolkit/8-1.png)
<center>图8-1 闭环分析型客户关系管理</center>

在其他环境中，信息必须以一种更实时的基础来反馈到操作型网站或呼叫中心。在此情况下，闭环比图8-1所示的情况有更紧密的方式，因为这种方式要收集并存储，然后反馈到集成系统中  
显然，组织越来越关注客户，也越来越需要DW/BI系统。最重要的是，一致性客户维度的需求变得更加关键 5  
## 8.2 客户维度属性
一致性客户维度是建立高效客户关系管理的关键元素  
客户维度通常是所有DW/BI系统维度中最具挑战性的维度。更为复杂的情况是，客户维度通常标识的是融合了多个内部和外部源系统的集成数据  
下一节，我们将关注多种客户维度设计考虑。从客户处获得的描述性信息越多，客户维度就越健壮，分析就更有意思  
### 8.2.1 名字与地址的语法分析
无论您处理的是个体的人还是商业实体，通常都需要获取客户姓名和地址属性。考察图8-2所示的用一般方法建立的列  
![8-2](../syn_pic/the_datawarehouse_toolkit/8-2.png)
<center>图8-2 过于一般化的客户姓名/地址示例数据 5</center>

这样设计的姓名/地址数据列将会受到太多限制。很难采用一致的机制处理称谓、标题和前缀  
在示例的地址属性中，不同地方采用的缩写形式不一  
与其使用通用意义的列，不如将姓名和地址属性拆分为多个可能的部分。抽取过程需要针对原先混乱的姓名和地址进行语法分析  
图8-3展示的是美国人地址和姓名属性。为方便理解，每个属性都包含一个示例数据，但真实的示例与展现的不同  
![8-3](../syn_pic/the_datawarehouse_toolkit/8-3.png)
<center>图8-3 包含姓名和地址语法分析的示例客户姓名/地址数据 5</center>

商业客户可能包含多个地址，例如，实体工厂地址和商店地址，每个地址都应该遵守图8-3展示的地址结构的逻辑含义  
### 8.2.2 国际姓名和地址的考虑
国际化展示和打印通常需要表示外文字符，不仅包括来自西欧的重音字符，也包括斯拉夫语、阿拉伯语、日语和中文，以及其他一些并不为人所熟悉的书写系统  
国际系统构建组织Unicode协会定义了一个称为Unicode的标准，用于表示世界上几乎所有国家的语言和文化所涉及的字符和字母  
需要注意的是，实现Unicode解决方案位于系统的基础层  
除了操作系统，所有用于获取、存储、转换和打印字符的设备都必须支持Unicode 5  
如果要处理的客户来自多个国家，客户地理属性会变得相当复杂  
注意：有时客户维度可能包括完整的地址块属性。该属性可用于那些有当地特色的国际位置  
**国际DW/BI目标**  
讨论了Unicode基础后，除了前面讨论的名称和地址分析需求以外，还需要牢记以下目标：  
+ **通用型和一致性**。如果打算使设计系统能够适合国际环境，让它能够在世界各地工作，则需要仔细考虑，BI工具是否产生多种语言的报表转换版本  
    + 排序序列存在差异，因此要么报表按照不同方式排序，要么除了那些采用“基本”语言的报表，其他所有报表都讲以无序方式出现 5 
    + 如果属性粒度在跨月宴环境下未能忠实保留，则要么分组总计将会出现差异，要么不同的语言的某些分组将包含看起来不对的重复行表头  
    + 所有BI工具的消息和提示符需要转换以方便用户使用  
+ **端到端数据质量以及下游的兼容性**  
    在整个数据流程中，数据仓库不是唯一需要考虑国际化姓名和地址的地方  
    从数据清洗和存储步骤开始，到最后一步执行地理和人口统计分析及打印报表步骤的整个过程中，都需要提供设计方面的考虑，以实现对获取名称和地址的支持  
+ **文化的正确性**  
    多数情况下，国外客户和合作伙伴将以某种方式获取DW/BI系统的最终结果。如果您不知道哪个名称的哪个部分是姓，哪个部分是名，不知道如何称呼人，那么您将会冒不尊重他人的风险，或者至少，看起来非常愚蠢 5  
+ **实时客户响应**  
    DW/BI系统可以通过支持实时客户响应系统，扮演一个操作型角色。客户服务代理可以回答电话，或者在不多于5秒左右的等待后从屏幕上得到一个数据仓库推荐使用的问候  
+ **其他类型的地址**  
    我们正置身与一场通信与网络的革命中。如果设计的系统能够处理国际姓名和地址，则必须预选考虑处理电子姓名、安全标志和网络地址  
    
### 8.2.3 客户为中心的日期
客户维度通常包含多种日期，例如，首次购买日期、最近一次购买的日期、生日等等  
尽管这些日期最初可能是SQL日期类型的列，但如果希望根据特定的日历属性汇总这些日期（例如，按照季节、季度、财务周期等等），则这些日期必须转变为引用日期维度的外键  
如图8-4所示的设计一种维度支架表，将在8.2.7小节讨论 5  
![8-4](../syn_pic/the_datawarehouse_toolkit/8-4.png)
<center>图8-4 日期维度支架表</center>

### 8.2.4 作为维度属性的聚集事实
商业用户通常喜欢基于聚集的性能度量约束客户维度，例如，从所有用户中过滤出那些在上一年度中花费超过一定数额的客户。将聚集事实作为维度属性提供给用户，将会使客户满意  
虽然存储这些属性可获得查询的可用性和改进性能，但主要的负担都落到ETL过程中，ETL过程需要确保属性的精确性，确保它们是最新的，并与实际的事实表行保持一致  
与其将属性存储为美元值，不如将它们替换为描述值，例如下一节将谈论的高额购买者。这些描述值减少了无法将数字属性与适当事实表联系的脆弱性  
此外，要保证所有用户对高额购买者有一致的定义，例如。不要使用自己定义的业务规则 5  
### 8.2.5 分段属性与记分
客户维度中最强有力的属性是分段类。对某个个体来说，肯能包括:  
+ 性别
+ 民族
+ 年龄或其他生命分段方式
+ 收入或其他生活类型分类 5
+ 状态（例如，新客户、活跃客户、不活跃客户、已离去客户）  
+ 参考源
+ 特定业务市场分段（例如，优先客户标识符）

类似地，许多组织为客户打分以刻画客户情况。每个客户用所得的分数标记  
1. 行为标记时间序列  
    一个常用的客户评分及分析系统方法是考察客户行为的相关度(R)、频繁度（F)和强度（I)。该方法被称为RFI方法 5  
    在处理大型客户数据库时，某个客户的行为可以按照如图8-5所示的RFI多维数据库建模  
    ![8-5](../syn_pic/the_datawarehouse_toolkit/8-5.png)
    <center>图8-5 相关度、频繁度和强度（RFI)多维数据库</center>

    如果在多维数据库中有上百万个点，则要理解不同分组的含义就比较困难。数据挖掘专家可能会利用下列的行为标识，更复杂一点的场景可能会包含信用行为和回报情况：  
    1. A高容量常客户，信誉良好，产品回报一般
    2. B高容量常客户，信誉良好，产品回报多 5
    3. C最近的新用户，尚未建立信誉模式
    4. D偶尔出现的客户，信誉良好
    5. E偶尔出现的客户，信誉不好
    6. F以前的优秀客户，最近不常见
    7. G只逛不买的客户，几乎没有效益
    8. H其他客户 5  
    
   至此可以考察客户时间序列数据并将某个客户关联到报表期间的最近分类中。这样，在最近10个考察期间，名为约翰的客户情况可以表示为：  
   约翰：C C C D D A A A B B  
   这一行为时间序列标记是不寻常的，因为它来自于固定周期度量过程，观察值是文本类型的。行为标记不是数字型的，不能计算或求平均值，但是它们可以被查询  
   行为标记可能不会被当成普通事实存储。行为标记的主要作用在于为类似前一段描述的例子制定复杂的查询模式。如果行为标记被存储在不同的事实表行中，此类查询将非常困难，需要串联关联的子查询  
   推荐的处理行为标记的方法是在客户维度中建立属性的时间序列。这也是位置设计的一个例子 5  
   除了为每个行为标记时间周期建立不同的列，建立单一的包含将所有行为标记连在一起的属性也是非常好的一种方法，例如，CCCDDAAABB。该列支持通配符搜索外部模式，例如，“D后紧跟着B”  
   注意：除了客户维度时间序列行为标记外，在微型维度中包含一个当前行为标示值也是非常合理的  
2. 数据挖掘与DW/BI之间的关系  
    数据挖掘小组是数据仓库的重要客户，是客户行为数据的特别重要的用户。然而，数据仓库用户发布数据的速度与数据挖掘用户使用数据的速度存在不匹配的情况  
    数据挖掘小组可能会喜欢这样的数据！例如，包含上百万此类查询结果的大型文件可以用决策树工具分析，该工具定位于Total Purchases Lifetime列，如上加粗文字  
    但是数据挖掘小组希望反复使用此类查询结果用于不同种类的分析工具，可能使用人工神经网络或者基于示例的推理工具 5  
    与其反复为数据仓库小组建立复杂的查询来生产庞大、昂贵的查询结果，不如将查询结果集合写入一个文件中，让数据挖掘小组在他们自己的服务器中分析  
    
### 8.2.6 包含类型2维度变化的计算
在业务上通常希望基于客户的属性，而不必与事实表连接来实现对客户的计算。如果采用类型2跟踪客户维度变化，则需要注意避免重复计算  
如果需要针对客户的历史事件，使用客户中的有效日期和失效日期进行计算，则情况会更加复杂。在本例中，不再有合法的客户行的失效日期行比新建立行的有效日期要少1天  
### 8.2.7 低粒度属性集合的支架表
在第3章中，我们鼓励设计者避免使用雪花模式，雪花模式将维度中低粒度的列放入不同的规范化表中，然后将这些规范化表与原始维度表关联  
除了这一针对雪花模式的限制以外，也存在一些特殊情况，允许建立维度支架表，支架表看起来与雪花模式有些类似 5  
如图8-6所示，维度支架表是来自外部数据提供者的数据集合，包含150个与客户居住的县有关的人口与社会经济属性  
![8-6](../syn_pic/the_datawarehouse_toolkit/8-6.png)
<center>图8-6 分类低粒度属性的维度支架表</center>

支持“不采用雪花模式”规则的原因在于，首先，人口统计数据与主维度数据比较存在几种不同的粒度，并且它没有多大的分析价值。与客户维度的其他数据比较，它在不同的时间被多次加载  
另外，如果基本客户维度非常大，则采用该方法可以节省大量的空间  
警告：可以使用维度支架表，但只可偶一为之，不要常用 5  
### 8.2.8 客户层次的考虑
处理商业客户问题最具有挑战性的问题之一是建模其组织的内部层次，这些层次关系可能会因为客户内部重组或者参与收购与资产剥离而经常发生变化  
注意：第7章描述了如何处理固定层次、稍有变化的层次和参差不齐的可变深度层次  
尽管不那么常见，但我们中的一些幸运儿有时能遇到客户层次具有高度可预测固定层次的情况。这就是我们在第7章中谈到的固定深度层次方法  
与之不同的另外一个客户具有1个、2个和3个层次的组合情况，可以将低层值复制到高层属性中。这就是第7章所讨论的稍有变化的层次  
多数情况下，复杂的商业客户层次都具有无法确定层次深度和参差不齐的特性，因此需要采用第7章介绍的参差不齐的可变深度层次建模技术 5  
## 8.3 应用于多值维度的桥接表
维度建模的一个基本原则是确定事实表粒度，然后仔细将维度和事实表增加到设计中，保持同样的粒度。但是也可能存在一些“问题”  维度，它们在个体交易粒度方面存在多个值。常见的多值维度示例包括：  
+ 来自多源的人口统计学描述
+ 某个商业客户的联系地址
+ 求职者的职业技能
+ 某个人的爱好 5
+ 病人的症状和诊断
+ 汽车或卡车的可选功能
+ 银行账户的联名持有人
+ 出租房的房客

面对多值维度，有两个基本选择：位置设计或桥接表设计。位置设计非常有吸引力，因为多值维度传播到易于查询的命名列中 5  
位置设计方法难以扩展，很快就会将数据库中的列用光了，要增加新列很困难，如果您设计的列包含所有爱好，则任何一个单一个体的维度行将会包含大量的空值  
采用桥接表方法处理多值维度是一种非常强有力的方法，但也伴随着较大的争议  
警告：使用桥接表的复杂查询可能需要超越BI工具范围之外的SQL  
接下来的两小节，将讨论适合本章以客户为中心主题的多值桥接表设计  
### 8.3.1 稀疏属性的桥接表
很多机构面临收集越来越多的有关客户的人口统计和状态信息，处理此类属性传统上锁采用的固定列建模方法在面对成百上千的属性时显得力不从心 5  
位置设计为每个属性建立命名列。列数据库比较适合此类设计，因为新列可以方便地以最小的中断方式加入到内部数据的存储中，包含较少的离散值的低粒度列可以被大幅度地压缩  
当不同属性的数量不断增长超过了适宜范围，并且新属性增加非常频繁时，我们推荐使用桥接表。例如，假设您将贷款申请信息收集起来，包含一个开放的名称-值对，如图8-7所示  
![8-7](../syn_pic/the_datawarehouse_toolkit/8-7.png)
<center>图8-7 贷款申请名称-值对数据</center>

一段时间后，可能收集到上百个甚至上千个不同的贷款申请变量。在此情况下，无论何时，变量的数量是开放的，无法预测的，此时采用如图8-8所示的桥接表方法非常合适  
![8-8](../syn_pic/the_datawarehouse_toolkit/8-8.png)
<center>图8-8 应用于大量不同种类、稀疏名称-值对数据集合的桥接表 5</center>

### 8.3.2 应用于客户多种联系方式的桥接表  
大多数商业客户具有多种联系方式，包括决策者、购买代理、部门主管、客户联络人等。由于联系数量不可预测且比较繁多，桥接表设计是处理这种情况的方便方法，如图8-9所示  
![8-9](../syn_pic/the_datawarehouse_toolkit/8-9.png)
<center>图8-9 多种联系方式的桥接表设计 5</center>

## 8.4 复杂的客户行为
客户行为可能非常复杂。本节中，我们将讨论处理顾客队列组和获取连续地行为。同时还将包括事实表的准确时间范围问题和用客户满意度或异常情况的指标标注事实事件  
### 8.4.1 客户队列的行为研究分组  
在分析客户时，类似在某个地理区域种上一年向客户卖出多少产品这样的简单查询快速发展到类似在上月有多少客户的购买量比他们在上一年的平均购买量要多这样的复杂查询  
某些情况下，您可能希望从某个查询获取客户集合或异常情况报告，例如，去年最出色的100个客户，上个月消费超过1000美元的客户，或接受了特殊测试要求的客户，然后使用客户分组（该分组也称为行为研究分组），不需要重新处理就可开展后续的分析工作  
通过利用客户的持久键，研究分组维度不会受制于客户维度的类型2变化，这种变化可能会在研究分组成员被确定后发生 5  
注意：建立复杂行为研究分组查询的秘密在于获取您需要跟踪的客户或产品行为的主键  
无论何时您希望约束任何表的所有分析，将它们获取到特定客户集合，现在您都可以使用这一客户主键特殊行为研究分组维度表  
唯一的需求时事实表包含客户主键引用。行为研究分组维度的使用如图8-10所示  
![8-10](../syn_pic/the_datawarehouse_toolkit/8-10.png)
<center>图8-10 通过客户维度持久性键与客户维度连接的行为研究分组维度</center>

行为研究分组维度附带一个与客户维度持久性键（参考图8-10所示的Customer ID)的等值连接 5  
注意：由于研究分组表异常简单，可以对它们采用联合、交集、并集和差集等操作  
通过将发生日期作为与每个持久键关联的第二个列，研究分组将变得更加强大  
与其他设计决策类似，这种策略表示了一定的妥协。这些研究分组表必须与主事实表在同一个空间，因为它们将被直接连接到客户维度表上  
### 8.4.2 连续行为的步骤维度  
多数DW/BI系统都有实现连续过程的良好示例。通常，度量获取于特定为止，用于考察用户流或产品动向  
通过介绍步骤维度，您可以将个体步骤放入整个会话的环境种，如图8-11所示 5  
![8-11](../syn_pic/the_datawarehouse_toolkit/8-11.png)
<center>图8-11 获取连续活动的步骤维度</center>

步骤维度是一个事前定义的抽象维度。维度中的第1行只能用于一个步骤的会话，其中当前步骤是第1步并且没有其他的剩余步骤  
步骤维度中第1行后紧接的两行用于两步的会话。其中的第1行（步骤键=2）是步骤号1，包含不止一个步骤，下一行（步骤键=3）步骤号为2，该步骤后没有其他步骤  
步骤维度可以被重建以方便可能包含100个步骤的会话。从图8-11中，您可以发现步骤维度可以于事务事实表关联，其中事实表的粒度是个体页面事件。在该例中，步骤维度有三种角色  
第1个角色是整个会话。第2个角色是连续的购买子会话，其中页面事件的序列产生确认的购买。第3个角色是被遗弃的购物车，购买没有发生时，页面事件的序列被终止 5  
使用步骤维度，特定的某个页可以立即被放入1个或多个能够被理解的环境中（整个会话、连续购买、抛弃的购物车）  
更为有趣的是，查询可以唯一地被约束至连续购买的首页。这是一个经典的Web事件查询，其中连续会话的“引诱”页被确定  
相反，查询可以被唯一约束到抛弃购物车的最后一页，此处用户将决定去哪里  
建模连续行为的另外一种方法是为每个可能出现的步骤建立特殊的固定编码  
如果需要跟踪零售环境中的客户产品购买行为，如果每个产品可以被编码，例如，以5位数字号码编码，那么您能够为每个客户建立包含产品代码序列的文本列 5  
使用非数字的字符来分割代码。这一的系列如下所示  
11254|45882|53340|74934|21399|…  
现在使用通配符可以搜素特定的产品购买序列或于其一起购买的产品的情况，或者在卖出某个产品的同时，另外某个产品未被卖出的情况  
### 8.4.3 时间范围事实表
在大量操作型应用中，可能希望检索客户在过去任意时刻的确切状态。若仔细管理包含客户事件的交易事实表，上述所有问题都能回答  
关键的建模步骤是包括一对日期/时间戳，如图8-12所示。第1个日期/时间戳是事务的准确事件，第2个日期/事件戳是另外一个事务的准确时间 5  
客户事务的时间历史将维护一个无缝的日期/时间戳的连续序列，每个实际事务保证您能够关联到客户人口统计信息和状态信息  
稠密的事务事实表非常有意思，因为您可以潜在地在事务发生时改变人口统计，特别时状态信息  
![8-12](../syn_pic/the_datawarehouse_toolkit/8-12.png)
<center>图8-12 时间范围事实表的双日期/时间戳</center>

关键的理解在于给定事务的日期/时间戳对定义了时间范围，其人口统计和状态是常量。查询可以利用此类“静态”的时间范围  
这样的日期/时间戳可用于针对客户库的棘手的查询。如果希望发现所有的在2013年曾经处于欺诈警告的客户，可以编写如下的SQL语句 5  
令人惊奇的是，该查询处理所有跨越2013年的开始和结束有效日期/时间的情况，可能包含在2013年中，或者完全跨越2013年  
也可以计算每个客户在2013年间处于欺诈警告的天数：  
**双重日期/时间戳的后端管理**  
对某个给定的客户，事务序列中的日期/时间戳必须构成完美的无缝序列  
当某个新事务行加入时，使用成对日期/时间戳需要两步过程  
第1步，当前事务的结束有效日期/事件戳必须被设置为未来的虚拟日期/时间 5  
第2步，在将新事务插入数据库后，ETL过程必须检索先前的事务并设置其结束有效日期/时间为最新插入的日期/时间  
### 8.4.4 使用满意度指标标记事实表  
在多数机构中，盈利是最重要的关键性能指标，客户满意度通常是处于第2位的指标  
满意度类似于盈利指标，需要集成多种资源  
实际上，每个面向过程的客户都是满意度信息的潜在来源之一，无论这一资源是销售、退货、客户支持、计费、网上活动、社会媒介、甚至是地理定位数据  
满意度数据可以是数值，也可以是文本。即时度量可以是可加的数值事实，也可以是服务级别维度的文本属性 5  
图8-13展示了常旅客满意度维度，该维度可以被加到第12章所描述的飞行活动事实表中。文本满意度数据一般以两种方式建模，主要看满意度属性的数量和输入数据的稀疏性  
当满意度属性的列表有界且相当稳定时，采用图8-13所示的位置设计是非常有效的  
![8-13](../syn_pic/the_datawarehouse_toolkit/8-13.png)
<center>图8-13 航空领域常旅客位置满意度维度</center>

### 8.4.5 使用异常情景指标标记事实表
累积快照事实表依赖一系列实现流水线过程的“标准场景”的日期。对订单实现来说，包含的步骤涉及订单建立、订单发货、订单交付、订单支付和订单退货等标准步骤  
但是如果偶尔出现偏离正常的情况时，没有号的办法用于揭示发生了什么情况。在累积快照的标准场景中往往没有考虑对此类情况建模 5  
描述针对标准情况的异常情况的方法是在累积快照事实表上增加一个发送状态维度。针对此类怪异的交货场景，使用状态怪异标记该订单完成列 5  
## 8.5 客户数据集成方法  
典型的环境通常是很多客户面对过程，此时需要对两个方法加以选择：从所有客户源系统记录的多种版本中获取单一的客户维度或者通过一致性维度联系起来的多个客户维度  
### 8.5.1 建立单一客户维度的主数据管理  
